In [1]:
from bs4 import BeautifulSoup


In [2]:
from playwright.async_api import async_playwright

In [3]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)

In [4]:
page = await browser.new_page()

In [5]:
await page.goto("https://eemis.mglsd.go.ug/jobs")

<Response url='https://eemis.mglsd.go.ug/jobs' request=<Request url='https://eemis.mglsd.go.ug/jobs' method='GET'>>

In [6]:
#await page.locator('.load-more').click()

In [7]:
while True:
    try:
        await page.locator('.load-more').click()
        print("Clicked 'Load More' button.")
        await page.wait_for_timeout(20000)  
    except Exception as e:
        print("No more 'Load More' button or an error occurred:", e)
        break

Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
Clicked 'Load More' button.
No more 'Load More' button or an error occurred: Locator.click: Timeout 30000ms exceeded.
Call log:
waiting for locator(".load-more")



In [8]:
html = await page.content()

In [9]:
soup_doc = BeautifulSoup(html)
#print(soup_doc.prettify())

In [10]:
all_jobs_html = soup_doc.find_all('li')[6:-5]
#all_jobs_html

In [11]:
jobs_page_list = []

for job in all_jobs_html:
    job_dict = {}
    #print(job.find('a').text)
    job_name = job.find('a').text
    job_dict['job_name_jobs_list'] = job_name
    #print(job.find('p').text)
    job_company = job.find('p').text
    job_dict['job_company_jobs_list'] = job_company
    #print(job.find('a')['href'])
    job_page_link = job.find('a')['href']
    job_dict['job_page_link'] = job_page_link
    #print(job.find('i').next)
    jobs_list_country = job.find('i').next
    job_dict['jobs_country_jobs_list'] = jobs_list_country
    #print(job.find(class_='fa fa-calendar').next)
    posted_date = job.find(class_='fa fa-calendar').next
    job_dict['posted_date'] = posted_date
    #print(job.find_all('i')[2].next)
    deadline = job.find_all('i')[2].next
    job_dict['deadline'] = deadline
    #print(job.find(class_='fa fa-money').next)
    salary = job.find(class_='fa fa-money').next
    job_dict['job_salary_jobs_list'] = salary
    jobs_page_list.append(job_dict)

jobs_page_list[0:5]

[{'job_name_jobs_list': 'HOUSEMAIDS',
  'job_company_jobs_list': 'HAM PROPERTY SERVICES AND MANAGEMENT LTD',
  'job_page_link': './jobs?id=6615',
  'jobs_country_jobs_list': 'Uganda',
  'posted_date': 'Posted : 02 Jan, 2025',
  'deadline': 'Deadline : 29 Jan, 2025',
  'job_salary_jobs_list': 'SAR 900'},
 {'job_name_jobs_list': 'housemaid',
  'job_company_jobs_list': 'INTERNATIONAL EMPLOYMENT LINKAGES (U) LTD',
  'job_page_link': './jobs?id=6614',
  'jobs_country_jobs_list': 'Uganda',
  'posted_date': 'Posted : 02 Jan, 2025',
  'deadline': 'Deadline : 29 Jan, 2025',
  'job_salary_jobs_list': 'SAR 900'},
 {'job_name_jobs_list': 'Sales',
  'job_company_jobs_list': 'MALISU APOLLO INVESTMENTS LTD',
  'job_page_link': './jobs?id=6613',
  'jobs_country_jobs_list': 'Uganda',
  'posted_date': 'Posted : 03 Jan, 2025',
  'deadline': 'Deadline : 26 Jan, 2025',
  'job_salary_jobs_list': 'AED 1200'},
 {'job_name_jobs_list': 'HOUSEMAIDS',
  'job_company_jobs_list': 'Peak Labour Giants Limited',
  'jo

## Scraping one job page

In [12]:
import re 

jobs_page_list[0]['job_page_link'].split('.')[-1]

'/jobs?id=6615'

In [13]:
import requests

html = requests.get("https://eemis.mglsd.go.ug/jobs?id=6570").content

In [14]:
soup_try = BeautifulSoup(html, "html.parser")

In [15]:
soup_try.find(class_="text-col").find('h2').find('a').text

'Static Guard/Security'

In [16]:
soup_try.find(class_="text-col").find('p').text

'Enjaazi Limited'

In [17]:
soup_try.find(class_="text-col").find('i').next

'Iraq'

In [18]:
soup_try.find(class_="text-col").find(class_="fa fa-calendar").next

'Dec 06, 2024 - Dec 06, 2025 '

In [19]:
soup_try.find(class_="text-col").find(class_="fa fa-money").next

'USD 1350'

In [20]:
soup_try.find(class_="clearfix").find('a').text

'Security'

In [21]:
soup_try.find('h4').next_sibling

'We are looking for dedicated and professional security guards to join our team in protecting key international diplomatic locations. The ideal candidate will have prior military experience, particularly with the UPDF, private security, and be ready to undergo thorough vetting by foreign governments'

In [22]:
soup_try.find_all('h4')[1].next_sibling

'We are looking for dedicated and professional security guards to join our team in protecting key international diplomatic locations. The ideal candidate will have prior military experience, particularly with the UPDF, private security, and be ready to undergo thorough vetting by foreign governments\t\t\t\t\t\t'

In [23]:
soup_try.find(class_="sidebar").find(class_="text-box").find_all('p')[0].text

'06 December, 2024'

In [24]:
soup_try.find(class_="sidebar").find(class_="text-box").find_all('p')[1].text

'+256-707-681801'

In [25]:
address = soup_try.find(class_="sidebar").find(class_="text-box").find_all('p')[2:]

one_address_list = []

for line in address:
    #print(line.text)
    one_address_list.append(line.text)

one_address_list

['P O Box  5328,',
 'Plot 2, Fr Seryazi Nicholas Close, Makindye,',
 'Kampala,',
 'Uganda.']

## Scraping all job pages

In [26]:
import time

for job in jobs_page_list:
    time.sleep(2)
    url_end = job['job_page_link'].split('.')[-1]
    url = "https://eemis.mglsd.go.ug" + url_end
    job['job_page_full_url'] = url
    #print(url)
    try: 
        html = requests.get(url).content
        soup_doc = BeautifulSoup(html, "html.parser")
        job_title = soup_doc.find(class_="text-col").find('h2').find('a').text
        job['job_name_job_page'] = job_title
        company_name = soup_doc.find(class_="text-col").find('p').text
        job['company_name_job_page'] = company_name
        job_country = soup_doc.find(class_="text-col").find('i').next
        job['job_country_job_page'] = job_country
        job_time = soup_doc.find(class_="text-col").find(class_="fa fa-calendar").next
        job['job_time_job_page'] = job_time
        job_salary = soup_doc.find(class_="text-col").find(class_="fa fa-money").next
        job['job_salary_job_page'] = job_salary
        job_category = soup_doc.find(class_="clearfix").find('a').text
        job['job_category'] = job_category
        job_overview = soup_doc.find('h4').next_sibling
        job['job_overview'] = job_overview
        job_description_details = soup_doc.find_all('h4')[1].next_sibling
        job['job_description_details'] = job_description_details
        company_reg_date = soup_doc.find(class_="sidebar").find(class_="text-box").find_all('p')[0].text
        job['company_reg_date'] = company_reg_date
        company_phone = soup_doc.find(class_="sidebar").find(class_="text-box").find_all('p')[1].text
        job['company_phone'] = company_phone
        company_address = soup_doc.find(class_="sidebar").find(class_="text-box").find_all('p')[2:]
        single_address_list = []
        for line in address:
            single_address_list.append(line.text)
        job['company_address']
    except:
        print("ERROR")
        print(url)

jobs_page_list[0:5]

ERROR
https://eemis.mglsd.go.ug/jobs?id=6615
ERROR
https://eemis.mglsd.go.ug/jobs?id=6614
ERROR
https://eemis.mglsd.go.ug/jobs?id=6613
ERROR
https://eemis.mglsd.go.ug/jobs?id=6612
ERROR
https://eemis.mglsd.go.ug/jobs?id=6611
ERROR
https://eemis.mglsd.go.ug/jobs?id=6608
ERROR
https://eemis.mglsd.go.ug/jobs?id=6607
ERROR
https://eemis.mglsd.go.ug/jobs?id=6606
ERROR
https://eemis.mglsd.go.ug/jobs?id=6601
ERROR
https://eemis.mglsd.go.ug/jobs?id=6600
ERROR
https://eemis.mglsd.go.ug/jobs?id=6593
ERROR
https://eemis.mglsd.go.ug/jobs?id=6592
ERROR
https://eemis.mglsd.go.ug/jobs?id=6591
ERROR
https://eemis.mglsd.go.ug/jobs?id=6589
ERROR
https://eemis.mglsd.go.ug/jobs?id=6588
ERROR
https://eemis.mglsd.go.ug/jobs?id=6587
ERROR
https://eemis.mglsd.go.ug/jobs?id=6586
ERROR
https://eemis.mglsd.go.ug/jobs?id=6585
ERROR
https://eemis.mglsd.go.ug/jobs?id=6583
ERROR
https://eemis.mglsd.go.ug/jobs?id=6548
ERROR
https://eemis.mglsd.go.ug/jobs?id=6540
ERROR
https://eemis.mglsd.go.ug/jobs?id=6535
ERROR
http

[{'job_name_jobs_list': 'HOUSEMAIDS',
  'job_company_jobs_list': 'HAM PROPERTY SERVICES AND MANAGEMENT LTD',
  'job_page_link': './jobs?id=6615',
  'jobs_country_jobs_list': 'Uganda',
  'posted_date': 'Posted : 02 Jan, 2025',
  'deadline': 'Deadline : 29 Jan, 2025',
  'job_salary_jobs_list': 'SAR 900',
  'job_page_full_url': 'https://eemis.mglsd.go.ug/jobs?id=6615',
  'job_name_job_page': 'HOUSEMAIDS',
  'company_name_job_page': 'HAM PROPERTY SERVICES AND MANAGEMENT LTD',
  'job_country_job_page': 'Saudi Arabia',
  'job_time_job_page': 'Jan 02, 2025 - Jan 02, 2027 ',
  'job_salary_job_page': 'SAR 900',
  'job_category': 'Domestic Worker',
  'job_overview': 'HOUSEMAIDS',
  'job_description_details': 'HOUSEMAIDS\t\t\t\t\t\t',
  'company_reg_date': '11 December, 2024',
  'company_phone': '+256-414-252490'},
 {'job_name_jobs_list': 'housemaid',
  'job_company_jobs_list': 'INTERNATIONAL EMPLOYMENT LINKAGES (U) LTD',
  'job_page_link': './jobs?id=6614',
  'jobs_country_jobs_list': 'Uganda',


In [27]:
len(jobs_page_list)

32

In [28]:
import pandas as pd

df = pd.json_normalize(jobs_page_list)
df.head()

,job_name_jobs_list,job_company_jobs_list,job_page_link,jobs_country_jobs_list,posted_date,deadline,job_salary_jobs_list,job_page_full_url,job_name_job_page,company_name_job_page,job_country_job_page,job_time_job_page,job_salary_job_page,job_category,job_overview,job_description_details,company_reg_date,company_phone
0,HOUSEMAIDS,HAM PROPERTY SERVICES AND MANAGEMENT LTD,./jobs?id=6615,Uganda,"Posted : 02 Jan, 2025","Deadline : 29 Jan, 2025",SAR 900,https://eemis.mglsd.go.ug/jobs?id=6615,HOUSEMAIDS,HAM PROPERTY SERVICES AND MANAGEMENT LTD,Saudi Arabia,"Jan 02, 2025 - Jan 02, 2027",SAR 900,Domestic Worker,HOUSEMAIDS,HOUSEMAIDS\t\t\t\t\t\t,"11 December, 2024",+256-414-252490
1,housemaid,INTERNATIONAL EMPLOYMENT LINKAGES (U) LTD,./jobs?id=6614,Uganda,"Posted : 02 Jan, 2025","Deadline : 29 Jan, 2025",SAR 900,https://eemis.mglsd.go.ug/jobs?id=6614,housemaid,INTERNATIONAL EMPLOYMENT LINKAGES (U) LTD,Saudi Arabia,"Jan 02, 2025 - Jan 02, 2027",SAR 900,Domestic Worker,"Housemaid where her capacity of cleaning, wash...","Housemaid where her capacity of cleaning, wash...","28 February, 2024",+256-414-581258
2,Sales,MALISU APOLLO INVESTMENTS LTD,./jobs?id=6613,Uganda,"Posted : 03 Jan, 2025","Deadline : 26 Jan, 2025",AED 1200,https://eemis.mglsd.go.ug/jobs?id=6613,Sales,MALISU APOLLO INVESTMENTS LTD,United Arab Emirates,"Jan 03, 2025 - Jan 03, 2027",AED 1200,Sales/Business Development,The sales personnel will be responsible for An...,The sales personnel will be responsible for An...,"14 March, 2024",+256-773-001186
3,HOUSEMAIDS,Peak Labour Giants Limited,./jobs?id=6612,Uganda,"Posted : 02 Jan, 2025","Deadline : 18 Jan, 2025",SAR 900,https://eemis.mglsd.go.ug/jobs?id=6612,HOUSEMAIDS,Peak Labour Giants Limited,Saudi Arabia,"Jan 02, 2025 - Jan 02, 2027",SAR 900,Domestic Worker,House maids needed in Saudi Arabia to work for...,House maids needed in Saudi Arabia to work for...,"17 May, 2024",+256-701-588214
4,HOUSEMAID,DERM LOGISTICS UG LTD,./jobs?id=6611,Uganda,"Posted : 02 Jan, 2025","Deadline : 31 May, 2025",SAR 900,https://eemis.mglsd.go.ug/jobs?id=6611,HOUSEMAID,DERM LOGISTICS UG LTD,Saudi Arabia,"Jan 02, 2025 - Jan 02, 2027",SAR 900,Domestic Worker,This Job exists to assist households in carryi...,This Job exists to assist households in carryi...,"17 May, 2024",+256-772-403006


In [29]:
df.to_csv("jobs.csv", index=False)